# Модели VGG

## Импорт библиотек

In [1]:
import torch
import torch.nn as nn

## Реализация модели VGG 11

In [9]:
class VGG11(nn.Module):
    def __init__(self, num_classes=1000, dropout=0.5):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, (3, 3), padding=1), 
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )

        # self.avgpool = nn.AdaptiveAvgPool2d(7,7)

        self.flatten = nn.Flatten()

        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        # x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [10]:
vgg_11_model = VGG11()

inp_tensor = torch.rand([1, 3, 224, 224], dtype=torch.float32)
pred = vgg_11_model(inp_tensor)
print(pred.shape)

torch.Size([1, 1000])


## Реализация общего класса модели VGG

In [ ]:
cfgs = {
    "vgg_11":[],
    "vgg_13":[],
    "vgg_16":[],
    "vgg_19":[]
}

In [ ]:
class VGG11(nn.Module):
    def __init__(self, num_classes=1000, dropout=0.5):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, (3, 3), padding=1), 
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )

        # self.avgpool = nn.AdaptiveAvgPool2d(7,7)

        self.flatten = nn.Flatten()

        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        # x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

## Готовая архитектура VGG в torchvision